<h1>Table of Contents<span class="tocSkip"></span></h1>
<div class="toc"><ul class="toc-item"><li><span><a href="#Improvements-pursued-in-this-notebook" data-toc-modified-id="Improvements-pursued-in-this-notebook-1"><span class="toc-item-num">1&nbsp;&nbsp;</span>Improvements pursued in this notebook</a></span></li><li><span><a href="#MNIST-Basics:-Final-Code" data-toc-modified-id="MNIST-Basics:-Final-Code-2"><span class="toc-item-num">2&nbsp;&nbsp;</span>MNIST Basics: Final Code</a></span><ul class="toc-item"><li><span><a href="#With-SGD-&quot;by-hand&quot;" data-toc-modified-id="With-SGD-&quot;by-hand&quot;-2.1"><span class="toc-item-num">2.1&nbsp;&nbsp;</span>With SGD "by hand"</a></span></li><li><span><a href="#With-fastai.Learner" data-toc-modified-id="With-fastai.Learner-2.2"><span class="toc-item-num">2.2&nbsp;&nbsp;</span>With fastai.Learner</a></span></li></ul></li><li><span><a href="#Classify-all-10-digits" data-toc-modified-id="Classify-all-10-digits-3"><span class="toc-item-num">3&nbsp;&nbsp;</span>Classify all 10 digits</a></span></li></ul></div>

# Improvements pursued in this notebook


- Add RGB
- Change from binary classifier to multi category classifier:
    - add ims 0-9
    - add change loss fxn to cross entropy loss w/ softmax
    - change shape of final activation from 1 to 10
    - change label to 1HE

# MNIST Basics: Final Code

## With SGD "by hand"

In [ ]:
### Imports ###
from fastai.vision.all import *
from fastcore.test import *
from fastbook import plot_function


### Data ###
path = untar_data(URLs.MNIST_SAMPLE)
Path.BASE_PATH = path

# train
train3 = torch.stack( 
    [tensor(Image.open(o)) for o in (path/'train'/'3').ls()]         # Create list of 6131 28x28 im tensors, and
    ).float()/255                                                    ## stack them into one [6131,28,28] tensor.
train7 = torch.stack(
    [tensor(Image.open(o)) for o in (path/'train'/'7').ls()]         # Repeat for sevens.
    ).float()/255
train_ims = torch.cat([train3,train7]).view(-1, 28*28)               # Combine 3s a& 7s then reshape as [6131,786].
train_lbls = tensor([1]*len(train3) + [0]*len(train7)).unsqueeze(1)  # Create lbl tensors: 1 if im is 3 else 0.
train_ds = list(zip(train_ims,train_lbls))                           # Zip im,lbl to create dataset.
train_dl = DataLoader(train_ds, batch_size = 64*2*2*2, shuffle=True) # Create batches; create DataLoader (an iter).

# valid (not used until fastai.Learner)
valid3 = torch.stack(
    [tensor(Image.open(o)) for o in (path/'valid'/'3').ls()]
    ).float()/255
valid7 = torch.stack(
    [tensor(Image.open(o)) for o in (path/'valid'/'7').ls()]
    ).float()/255
valid_ims = torch.cat([valid3,valid7]).view(-1, 28*28)
valid_lbls = tensor([1]*len(valid3) + [0]*len(valid7)).unsqueeze(1)
valid_ds = list(zip(valid_ims,valid_lbls))
valid_dl = DataLoader(valid_ds, batch_size = 64*2*2*2, shuffle=True)


### Loss Function ###
def calibrated_confidence(yp, y, pctile=.95,top_activ=20):           # Loss function "Calibrated Confidence":
    pred = yp.sigmoid()                                              # - correct   & high confidence → low loss
    return torch.where(y==1, 1-pred, pred).mean()                    # - incorrect & high confidence → high loss


### Create Model & Initialize Params ###
three_layer_nn = nn.Sequential(                            # nn.Sequential() defines a model of composed fxns
    nn.Linear(28*28,30),                                   # nn.Linear.__init__(w,b) creates w,b with rand values
    nn.ReLU(),                                             # nn.Linear.__main__(t) returns t@w+b
    nn.Linear(30,1))                                       # nn.ReLU().__main__(t) returns t.max(tensor(0.))


### Create SGD Stepper ###
class ParamStepper:
    def __init__(self, p, lr): self.p,self.lr = list(p),lr # remembers your params & an lr
        
    def step(self, *args, **kwargs):                       # take one step in the optimal direction
        for o in self.p: o.data -= o.grad.data * self.lr
            
    def zero_grad(self, *args, **kwargs):                  # zeros out gradients
        for o in self.p: o.grad = None


### Calculate accuracy over one mini-batch given yp,y ###
def avg_batch_acc(yp,y):
    sig_yp = yp.sigmoid()
    correct = (sig_yp > 0.5) == y
    return correct.float().mean()


### Calculate accuracy over entire dl given dl,mod ###
def validate_epoch(dl, mod):                               # Given data `dl` and a model & params `mod`
    acc = [avg_batch_acc(mod(xb), yb) for xb,yb in dl]     # Gradients calculated & stored at mod(xb) call
    return round(torch.stack(acc).mean().item(), 5)        # Avg over all mini-batches, return scalar (not tensor)


### Adjust parameters w/ stepper for each mini-batch in a dl
def train_once(dl, mod, stepper):
    for xb,yb in dl:
        calc_grad(xb, yb, mod)
        stepper.step()
        stepper.zero_grad()


### Run `train_once` `n` times given data `dl`, model `mod`, and stepper `stepper`
def train_model(dl, mod, stepper, n):
    for i in range(n):
        train_once(dl, mod, stepper)
        print(validate_epoch(dl, mod), end='\t')

# together
sgd = ParamStepper(three_layer_nn.parameters(), lr=1.)
train_model(train_dl, three_layer_nn, sgd, 20)

## With fastai.Learner

In [ ]:
### Imports ###
from fastai.vision.all import *
from fastcore.test import *


### Data ###
path = untar_data(URLs.MNIST_SAMPLE)
Path.BASE_PATH = path

# train
train3 = torch.stack( 
    [tensor(Image.open(o)) for o in (path/'train'/'3').ls()]         # Create list of 6131 28x28 im tensors, and
    ).float()/255                                                    ## stack them into one [6131,28,28] tensor.
train7 = torch.stack(
    [tensor(Image.open(o)) for o in (path/'train'/'7').ls()]         # Repeat for sevens.
    ).float()/255
train_ims = torch.cat([train3,train7]).view(-1, 28*28)               # Combine 3s a& 7s then reshape as [6131,786].
train_lbls = tensor([1]*len(train3) + [0]*len(train7)).unsqueeze(1)  # Create lbl tensors: 1 if im is 3 else 0.
train_ds = list(zip(train_ims,train_lbls))                           # Zip im,lbl to create dataset.
train_dl = DataLoader(train_ds, batch_size = 64*2*2*2, shuffle=True) # Create batches; create DataLoader (an iter).

# valid
valid3 = torch.stack(
    [tensor(Image.open(o)) for o in (path/'valid'/'3').ls()]
    ).float()/255
valid7 = torch.stack(
    [tensor(Image.open(o)) for o in (path/'valid'/'7').ls()]
    ).float()/255
valid_ims = torch.cat([valid3,valid7]).view(-1, 28*28)
valid_lbls = tensor([1]*len(valid3) + [0]*len(valid7)).unsqueeze(1)
valid_ds = list(zip(valid_ims,valid_lbls))
valid_dl = DataLoader(valid_ds, batch_size = 64*2*2*2, shuffle=True)

dls = DataLoaders(train_dl, valid_dl)


## Model ###
three_layer_nn = nn.Sequential( # nn.Sequential composes fxns. Each fxn is a layer, ∴ this is a 3 layer nn.
    nn.Linear(28*28,30),        # nn.Linear creates linear parameters W and B as in Y = X@W+B.
    nn.ReLU(),                  # nn.Linear is a class. When called, it's __main__(x) function computes X@W+B.
    nn.Linear(30,1))            # nn.ReLU is the same as item-wise max(t, 0), which replaces all negs with 0s.


### Loss Function "Calibrated Confidence" ###
def loss(yp, y):                                  # I like to call this "Calibrated Confidence":
    pred = yp.sigmoid()                           # - correct   & high confidence → low loss
    return torch.where(y==1, 1-pred, pred).mean() # - incorrect & high confidence → high loss


### Mini-batch Average Accuracy given yp,y ###
def avg_batch_acc(yp,y):
    sig_yp = yp.sigmoid()
    correct = (sig_yp > 0.5) == y
    return correct.float().mean()


### Combine data, model, stepper, loss, accuracy in a Learner ###
learn = Learner(dls,                   # train and valid dls
                three_layer_nn,        # model
                opt_func=SGD,          # fastai.SGD optimizer
                loss_func=loss,        # loss fxn
                metrics=avg_batch_acc) # judgement metric


### Train ###
epochs = 40
lr = 0.1
learn.fit(epochs,lr)
plt.plot(L(learn.recorder.values).itemgot(2)) # L.itemgot(2) returns every 2nd row

Super-short version with all of the helpers:

In [ ]:
from fastai.vision.all import *

path = untar_data(URLs.MNIST_SAMPLE)
Path.BASE_PATH = path

dls = ImageDataLoaders.from_folder(path)
learn = cnn_learner(dls, resnet18, pretrained=False, loss_func=F.cross_entropy, metrics=accuracy)
learn.fit_one_cycle(1, 0.1)

epoch,train_loss,valid_loss,accuracy,time
0,0.056472,0.009414,0.998037,00:04


# Classify all 10 digits

In [ ]:
from fastai.vision.all import *

### Data ###
path = untar_data(URLs.MNIST)
n_cls = 10
batch_size = 64*2*2*2

# Train
# ims
for i in range(n_cls):
    new_ims = torch.stack(
        [tensor(Image.open(fn)) for fn in (path/'training'/f'{i}').ls()]
    ).float()/255
    if i == 0: ims = new_ims
    else: ims = torch.cat([ims,new_ims])
train_ims = ims.view(-1,28*28)
# lbls
train_lbls = []
for i in range(n_cls):
    l = L([0]*n_cls)
    l[i] = 1
    lbls += [l] * len((path/'training'/f'{i}').ls())    
train_lbls = tensor(lbls)

# Valid
# ims
for i in range(n_cls):
    new_ims = torch.stack(
        [tensor(Image.open(fn)) for fn in (path/'testing'/f'{i}').ls()]
    ).float()/255
    if i == 0: ims = new_ims
    else: ims = torch.cat([ims,new_ims])
valid_ims = ims.view(-1,28*28)
# lbls
valid_lbls = []
for i in range(n_cls):
    l = L([0]*n_cls)
    l[i] = 1
    lbls += [l] * len((path/'testing'/f'{i}').ls())    
valid_lbls = tensor(lbls)

# DataLoaders
train_ds = L(zip(train_ims, train_lbls))
valid_ds = L(zip(valid_ims, valid_lbls))
train_dl = DataLoader(train_ds, batch_size=batch_size, shuffle=True)
valid_dl = DataLoader(valid_ds, batch_size=batch_size, shuffle=True)
dls = DataLoaders(train_dl, valid_dl)

In [ ]:
### Model ###
my_nn = nn.Sequential(
    nn.Linear(28*28,50),
    nn.ReLU(),
    nn.Linear(50,30),
    nn.ReLU(),
    nn.Linear(30,10))


### Loss & Accuracy ###
def softmax(t):
    if len(t.shape) == 1: return torch.exp(t) / torch.exp(t).sum()
    else:                 return torch.exp(t) / torch.exp(t).sum(dim=1, keepdim=True)

def loss(yp, y):
    yps = softmax(yp)
    return (1 - (y * yps).sum(dim=1, keepdim=True)).mean()

def acc(yp,y):
    yp_max,yp_i = torch.max(yp, dim=1, keepdim=True)
    y_max, y_i  = torch.max(y,  dim=1, keepdim=True)
    return (yp_i==y_i).float().mean()

### Create SGD Stepper ###
class ParamStepper:
    def __init__(self, p, lr): self.p,self.lr = list(p),lr # remembers your params & lr
        
    def step(self, *args, **kwargs):                       # take one step in the optimal direction
        for o in self.p: o.data -= o.grad.data * self.lr
            
    def zero_grad(self, *args, **kwargs):                  # zeros out gradients
        for o in self.p: o.grad = None

### Calculate accuracy over entire dl given dl,mod ###
def validate_epoch(dl, mod):                       # Given data `dl` and a model & params `mod`
    a = [acc(mod(xb), yb) for xb,yb in dl]         # Gradients calculated & stored at mod(xb) call
    return round(torch.stack(a).mean().item(), 5)  # Avg over all mini-batches, return scalar (not tensor)


### Adjust parameters w/ stepper for each mini-batch in a dl
def train_once(dl, mod, stepper):
    for xb,yb in dl:
        calc_grad(xb, yb, mod)
        stepper.step()
        stepper.zero_grad()


### Calculate gradients for use in train_once ###
def calc_grad(x,y,model):
    yp = model(x)
    ls = loss(yp,y)
    ls.backward()


### Run `train_once` `n` times given data `dl`, model `mod`, and stepper `stepper`
def train_model(dl, mod, stepper, n):
    for i in range(n):
        train_once(dl, mod, stepper)
        print(validate_epoch(dl, mod), end='\t')


### Train ###
dl = train_dl
sgd = ParamStepper(p=my_nn.parameters(), lr=.1)
train_model(dl, my_nn, sgd, 20)

0.185	0.21428	0.22531	0.13134	0.22749	0.31196	0.34463	0.37202	0.39915	0.41556	0.43454	0.43852	0.44263	0.44541	0.44966	0.45151	0.45441	0.45545	0.45853	0.46076	